In [10]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
import sys
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers


EPOCHS     = 200
BATCH_SIZE = 1000
VERBOSE    = 1
VALIDATION_SPLIT = 0.2 # how much TRAIN is reserved for VALIDATION
OPTIMIZER = tf.keras.optimizers.Adam()#tf.keras.optimizers.Adam()


O = 'Berlin'

def get_data():
    print(f"Loading raw data for {O}.")
    FILENAME =  f"/Users/sonat/Documents/DataSets/Spatial/{O}_maps_labels.npz"
    data     = np.load(FILENAME)
    labels   = data['labels']
    data     = data["maps"]
    
    print(f"Loaded a dataset with {data.shape} and {labels.shape[0]} labels")
    return data, labels
def flatten(data):
    print('Flattening data')
    print(f"Old size:{data.shape}")
    data = data.reshape((data.shape[0],data.shape[1]*data.shape[2]))
    print(f"New size:{data.shape}")
    return data

def normalize(data):
    print("Normalizing data")
    data = data / np.std(data,axis=0)
    if not np.any(np.isclose(np.std(data,axis=0),1)):
        sys.exit('Normalization didn''t work')
    return data

def center(data):
    print("Centering data")
    data = data - np.mean(data,axis=0)
    if not np.any(np.isclose(np.mean(data,axis=0),0)):
        sys.exit('Centering didn''t work')
    return data

def take_most_active(data):
    return data
 
    
def splitter(data,labels):

    print('Splitter called.')
    X_train, X_test, Y_train, Y_test = \
    train_test_split(data,labels, test_size=0.2, random_state=42)
    
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    TRAIN_SAMPLES = X_train.shape[0]
    TEST_SAMPLES  = X_test.shape[0]
    
    # cast
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    # One-hot representation of the labels.
    NB_CLASSES     = np.unique(Y_train).shape[0]
    print(NB_CLASSES)
    Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
    Y_test  = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)
    return X_train, X_test, Y_train, Y_test


def two_hidden_layer(data, labels,activation,N_HIDDEN=16):
    INPUT_SHAPE = (data.shape[1],)
    NB_CLASSES     = np.unique(labels).shape[0]
    print(f"Building Network with 2 hidden layers alike")
    print(f"Found {NB_CLASSES} classes")
    print(f"Found {INPUT_SHAPE} input size")
    model = models.Sequential()
    
    model.add(keras.layers.Dense(N_HIDDEN,input_shape=INPUT_SHAPE,name='dense_layer', activation='relu'))
    model.add(keras.layers.Dense(N_HIDDEN,name='dense_layer_2', activation='relu'))
    model.add(keras.layers.Dense(NB_CLASSES,name='linear', activation=activation))    
    return model



def logistic(data, labels,activation):
    INPUT_SHAPE = (data.shape[1],)
    NB_CLASSES     = np.unique(labels).shape[0]
    print(f"Building model Logistic Network alike")
    print(f"Found {NB_CLASSES} classes")
    print(f"Found {INPUT_SHAPE} input size")
    model = models.Sequential()
    
    model.add(keras.layers.Dense(NB_CLASSES,
                                 input_shape=INPUT_SHAPE,
                                 name='dense_layer', 
                                 activation=activation))
    
    return model

pipeline = { 
#             0: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,         
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : logistic,
#                    'model_args'   : {'activation':'sigmoid'}
#                 },
#             1: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,         
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : logistic,
#                    'model_args'   : {'activation':'softmax'}
#                 },
#             2: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : two_hidden_layer,
#                    'model_args'   : {'activation':'softmax','N_HIDDEN':16}
#                 },
#             3: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : two_hidden_layer,
#                    'model_args'   : {'activation':'softmax','N_HIDDEN':64}
#                 },
#             4: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : two_hidden_layer,
#                    'model_args'   : {'activation':'softmax','N_HIDDEN':8}
#                 },
#             5: #one-layer, ie logistric regression
#                   {'data_loader'  : get_data,
#                    'preprocessors': [flatten, normalize],
#                    'splitter'     : splitter,
#                    'model'        : two_hidden_layer,
#                    'model_args'   : {'activation':'softmax','N_HIDDEN':64}
#                 },
            6: #one-layer, ie logistric regression
                  {'data_loader'  : get_data,
                   'preprocessors': [flatten, center],
                   'splitter'     : splitter,
                   'model'        : two_hidden_layer,
                   'model_args'   : {'activation':'sigmoid','N_HIDDEN':64},
                   'optimizer'    : tf.keras.optimizers.Adam(learning_rate=0.1)
                },
            7: #one-layer, ie logistric regression
              {'data_loader'  : get_data,
               'preprocessors': [flatten, center],
               'splitter'     : splitter,
               'model'        : two_hidden_layer,
               'model_args'   : {'activation':'sigmoid','N_HIDDEN':64},
               'optimizer'    : tf.keras.optimizers.Adam(learning_rate=0.001)
                },
            8: #one-layer, ie logistric regression
              {'data_loader'  : get_data,
               'preprocessors': [flatten, center],
               'splitter'     : splitter,
               'model'        : two_hidden_layer,
               'model_args'   : {'activation':'sigmoid','N_HIDDEN':64},
               'optimizer'    : tf.keras.optimizers.Adam(learning_rate=1)
                },
            }
#                  {'model_fun':logistic,'data_fun': get_data,'preprocess_fun': preprop}


In [11]:
!rm -rf /tmp/tfdbg2_logdir
tf.debugging.experimental.enable_dump_debug_info('/tmp/tfdbg2_logdir', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
#run the pipeline:
for i in pipeline.keys():
    data,labels = pipeline[i]['data_loader']()

    for p_fun in pipeline[i]['preprocessors']:
        data = p_fun(data)

    X_train, X_test, Y_train, Y_test = pipeline[i]['splitter'](data,labels)
    
    kwargs  = pipeline[i]['model_args']
    model = pipeline[i]['model'](data,labels,**kwargs)
    OPTIMIZER = pipeline[i]['optimizer']
    # Compiling the model.
    model.compile(optimizer=OPTIMIZER,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    model.summary()
    
    log_dir = "/tmp/tfdbg2_logdir/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Training the model.
    model.fit(X_train, Y_train,
              batch_size=BATCH_SIZE, 
              epochs=EPOCHS,
              verbose=VERBOSE, 
              validation_split=VALIDATION_SPLIT,
              callbacks=[tensorboard_callback])

    #evaluate the model
    test_loss, test_acc = model.evaluate(X_test, Y_test)
    print('Test accuracy:', test_acc)

INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: /tmp/tfdbg2_logdir, tensor debug mode: FULL_HEALTH)
Loading raw data for Berlin.
Loaded a dataset with (21980, 30, 30) and 21980 labels
Flattening data
Old size:(21980, 30, 30)
New size:(21980, 900)
Centering data
Splitter called.
17584 train samples
4396 test samples
7
Building Network with 2 hidden layers alike
Found 7 classes
Found (900,) input size
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 64)                57664     
_________________________________________________________________
dense_layer_2 (Dense)        (None, 64)                4160      
_________________________________________________________________
linear (Dense)               (None, 7)                 455       
Total params: 62,279
Trainable params: 62,279
Non-trainable params: 0
_________________

InvalidArgumentError: Nan in summary histogram for: dense_layer/kernel_0 [Op:WriteHistogramSummary]